<a href="https://colab.research.google.com/github/HamdanXI/nlp_adventure/blob/main/802%20code/bert-score-evaluation-t5-small.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets bert-score

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.3 MB/s eta 0:00:00


In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from datasets import load_dataset
from bert_score import score
import torch

tokenizer = AutoTokenizer.from_pretrained("HamdanXI/t5-small-paradetox-1Token-split-masked")
model = AutoModelForSeq2SeqLM.from_pretrained("HamdanXI/t5-small-paradetox-1Token-split-masked")

dataset = load_dataset("s-nlp/paradetox")

In [4]:
max_token_length_input = max(len(tokenizer.encode(item)) for item in dataset["train"]["en_toxic_comment"])
max_token_length_label = max(len(tokenizer.encode(item)) for item in dataset["train"]["en_neutral_comment"])

print(f"Max token length for input: {max_token_length_input}")
print(f"Max token length for label: {max_token_length_label}")

if max_token_length_input > max_token_length_label:
    highest_length = max_token_length_input
else:
    highest_length = max_token_length_label

print(highest_length)

Max token length for input: 48
Max token length for label: 43
48


In [5]:
# Function to generate predictions
def generate_predictions(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, max_length=max_token_length_label, return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(**inputs)
    return [tokenizer.decode(output, skip_special_tokens=True) for output in outputs]

# Generate predictions
predictions = generate_predictions(dataset['train']['en_toxic_comment'])

# Compute BERT Score
P, R, F1 = score(predictions, dataset['train']['en_neutral_comment'], lang="en", rescale_with_baseline=True)

# Compute and print the average scores
print(f"Precision: {P.mean()}, Recall: {R.mean()}, F1 Score: {F1.mean()}")

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


KeyboardInterrupt: ignored